In [3]:
from datasets import load_dataset
from IPython.display import display, Markdown
from collections import defaultdict
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import polars as pl

In [5]:
dataset_pl = pl.read_parquet("../codecontests_cot_sft.parquet")

In [24]:
for i in range(10):
    display(Markdown('## Problem'))
    display(Markdown(dataset_pl[i]['problem'][0]))
    display(Markdown('## Solution'))
    display(Markdown(dataset_pl[i]['completions'][0][1]))

## Problem

There are n persons who initially don't know each other. On each morning, two of them, who were not friends before, become friends.

We want to plan a trip for every evening of m days. On each trip, you have to select a group of people that will go on the trip. For every person, one of the following should hold: 

  * Either this person does not go on the trip, 
  * Or at least k of his friends also go on the trip. 



Note that the friendship is not transitive. That is, if a and b are friends and b and c are friends, it does not necessarily imply that a and c are friends.

For each day, find the maximum number of people that can go on the trip on that day.

Input

The first line contains three integers n, m, and k (2 ≤ n ≤ 2 ⋅ 10^5, 1 ≤ m ≤ 2 ⋅ 10^5, 1 ≤ k < n) — the number of people, the number of days and the number of friends each person on the trip should have in the group.

The i-th (1 ≤ i ≤ m) of the next m lines contains two integers x and y (1≤ x, y≤ n, x≠ y), meaning that persons x and y become friends on the morning of day i. It is guaranteed that x and y were not friends before.

Output

Print exactly m lines, where the i-th of them (1≤ i≤ m) contains the maximum number of people that can go on the trip on the evening of the day i.

Examples

Input

4 4 2
2 3
1 2
1 3
1 4


Output

0
0
3
3


Input

5 8 2
2 1
4 2
5 4
5 2
4 3
5 1
4 1
3 2


Output

0
0
0
3
3
4
4
5


Input

5 7 2
1 5
3 2
2 5
3 4
1 2
5 3
1 3


Output

0
0
0
0
3
4
4

Note

In the first example, 

  * 1,2,3 can go on day 3 and 4. 



In the second example, 

  * 2,4,5 can go on day 4 and 5. 
  * 1,2,4,5 can go on day 6 and 7. 
  * 1,2,3,4,5 can go on day 8. 



In the third example, 

  * 1,2,5 can go on day 5. 
  * 1,2,3,5 can go on day 6 and 7. 

## Solution

To solve this problem, we need to determine the maximum number of phone numbers that can be formed using the given digits. A phone number must start with '8' and be exactly 11 digits long. Therefore, each phone number requires exactly one '8' and ten other digits.

### Reasoning Steps:

1. **Count the Digits**: First, count the number of '8's and the total number of other digits available.
2. **Determine the Maximum Number of Phone Numbers**: The maximum number of phone numbers is limited by the number of '8's and the number of other digits. Specifically, the number of phone numbers cannot exceed the number of '8's, and it also cannot exceed the total number of other digits divided by 10 (since each phone number requires 10 other digits).
3. **Calculate the Result**: The result is the minimum of the number of '8's and the total number of other digits divided by 10.

### Implementation:

1. **Count the '8's**: We can count how many '8's are in the string.
2. **Count the Other Digits**: We can count how many digits are not '8'.
3. **Calculate the Maximum Number of Phone Numbers**: The maximum number of phone numbers is the minimum of the number of '8's and the number of other digits divided by 10.

### Python Code:

```python
def max_phone_numbers(n, s):
    # Count the number of '8's
    count_8 = s.count('8')
    
    # Count the number of other digits
    count_other = n - count_8
    
    # Calculate the maximum number of phone numbers
    max_phones = min(count_8, count_other // 10)
    
    return max_phones

# Example usage:
# n = 11
# s = "00000000008"
# print(max_phone_numbers(n, s))  # Output: 1
```

### Explanation:

- **count_8**: This variable stores the number of '8's in the string `s`.
- **count_other**: This variable stores the number of digits that are not '8'.
- **max_phones**: This variable calculates the maximum number of phone numbers that can be formed, which is the minimum of the number of '8's and the number of other digits divided by 10.

The function `max_phone_numbers` returns the maximum number of phone numbers that can be formed using the given digits. If no phone number can be formed, it returns 0.

## Problem

...

To solve this problem, we need to maximize the sum of the power levels of the minions under Polycarp's control, given that he can only have up to `k` minions at any time. The key insight is that the order in which minions are summoned affects the final power levels because summoning a minion increases the power levels of all previously summoned minions by its `b_i` value.

### Approach
1. **Sorting Minions**: We need to determine the optimal order in which to summon the minions. One effective strategy is to sort the minions based on their `b_i` values in descending order. This way, minions that provide the largest boosts to previously summoned minions are summoned first.

2. **Summoning and Destroying**: After sorting, we can iterate through the minions and decide whether to summon them or not. Since we can only have `k` minions at any time, we need to manage the minions carefully. If we have already summoned `k` minions, we need to destroy one before summoning a new one. The minion to destroy should be the one that contributes the least to the total power.

3. **Tracking Power Levels**: We need to keep track of the current power levels of the minions under control. When a new minion is summoned, we update the power levels of all existing minions by adding the `b_i` value of the new minion.

4. **Output the Sequence**: Finally, we need to output the sequence of actions (summoning and destroying) that leads to the maximum total power.

### Solution Code
```python
def solve():
    import sys
    input = sys.stdin.read
    data = input().split()
    
    idx = 0
    T = int(data[idx])
    idx += 1
    
    for _ in range(T):
        n = int(data[idx])
        k = int(data[idx+1])
        idx += 2
        
        minions = []
        for i in range(n):
            a = int(data[idx])
            b = int(data[idx+1])
            minions.append((a, b, i+1))  # storing original index as i+1
            idx += 2
        
        # Sort minions by b in descending order
        minions.sort(key=lambda x: -x[1])
        
        # Initialize the list of current minions and their power levels
        current_minions = []
        total_power = 0
        actions = []
        
        for a, b, original_idx in minions:
            if len(current_minions) < k:
                # Summon the minion
                actions.append(original_idx)
                # Update the power levels of all current minions
                for i in range(len(current_minions)):
                    current_minions[i] += b
                # Add the new minion's power
                current_minions.append(a)
                total_power += a
            else:
                # Find the minion with the smallest power to destroy
                min_power = min(current_minions)
                min_index = current_minions.index(min_power)
                # Destroy the minion
                actions.append(-(current_minions[min_index]))
                # Remove it from the list
                total_power -= current_minions[min_index]
                del current_minions[min_index]
                # Summon the new minion
                actions.append(original_idx)
                # Update the power levels of all current minions
                for i in range(len(current_minions)):
                    current_minions[i] += b
                # Add the new minion's power
                current_minions.append(a)
                total_power += a
        
        # Output the number of actions and the actions themselves
        print(len(actions))
        print(' '.join(map(str, actions)))

# Call the function to solve the problem
solve()
```

### Explanation
1. **Reading Input**: The input is read and parsed into the number of test cases, followed by the number of minions and the maximum number of minions Polycarp can control for each test case.

2. **Sorting Minions**: The minions are sorted based on their `b_i` values in descending order to maximize the boost to previously summoned minions.

3. **Summoning and Destroying**: For each minion, if the number of current minions is less than `k`, it is summoned directly. If the number of minions is already `k`, the minion with the smallest power is destroyed before summoning the new one.

4. **Tracking Power Levels**: The power levels of the minions are updated whenever a new minion is summoned. The total power is also updated accordingly.

5. **Output**: The sequence of actions (summoning and destroying) is printed for each test case.

This approach ensures that the sum of the power levels of the minions under Polycarp's control is maximized.

## Problem

Polycarp plays a computer game. In this game, the players summon armies of magical minions, which then fight each other.

Polycarp can summon n different minions. The initial power level of the i-th minion is a_i, and when it is summoned, all previously summoned minions' power levels are increased by b_i. The minions can be summoned in any order.

Unfortunately, Polycarp cannot have more than k minions under his control. To get rid of unwanted minions after summoning them, he may destroy them. Each minion can be summoned (and destroyed) only once.

Polycarp's goal is to summon the strongest possible army. Formally, he wants to maximize the sum of power levels of all minions under his control (those which are summoned and not destroyed).

Help Polycarp to make up a plan of actions to summon the strongest possible army!

Input

The first line contains one integer T (1 ≤ T ≤ 75) — the number of test cases.

Each test case begins with a line containing two integers n and k (1 ≤ k ≤ n ≤ 75) — the number of minions availible for summoning, and the maximum number of minions that can be controlled by Polycarp, respectively.

Then n lines follow, the i-th line contains 2 integers a_i and b_i (1 ≤ a_i ≤ 10^5, 0 ≤ b_i ≤ 10^5) — the parameters of the i-th minion.

Output

For each test case print the optimal sequence of actions as follows:

Firstly, print m — the number of actions which Polycarp has to perform (0 ≤ m ≤ 2n). Then print m integers o_1, o_2, ..., o_m, where o_i denotes the i-th action as follows: if the i-th action is to summon the minion x, then o_i = x, and if the i-th action is to destroy the minion x, then o_i = -x. Each minion can be summoned at most once and cannot be destroyed before being summoned (and, obviously, cannot be destroyed more than once). The number of minions in Polycarp's army should be not greater than k after every action.

If there are multiple optimal sequences, print any of them.

Example

Input


3
5 2
5 3
7 0
5 0
4 0
10 0
2 1
10 100
50 10
5 5
1 5
2 4
3 3
4 2
5 1


Output


4
2 1 -1 5
1
2
5
5 4 3 2 1

Note

Consider the example test.

In the first test case, Polycarp can summon the minion 2 with power level 7, then summon the minion 1, which will increase the power level of the previous minion by 3, then destroy the minion 1, and finally, summon the minion 5. After this, Polycarp will have two minions with power levels of 10.

In the second test case, Polycarp can control only one minion, so he should choose the strongest of them and summon it.

In the third test case, Polycarp is able to summon and control all five minions.

## Solution

To solve this problem, we need to maximize the sum of the power levels of the minions under Polycarp's control, given that he can only have up to `k` minions at any time. The key insight is that the order in which minions are summoned affects the final power levels because summoning a minion increases the power levels of all previously summoned minions by its `b_i` value.

### Approach
1. **Sorting Minions**: We need to determine the optimal order in which to summon the minions. One effective strategy is to sort the minions based on their `b_i` values in descending order. This way, minions that provide the largest boosts to previously summoned minions are summoned first.

2. **Summoning and Destroying**: After sorting, we can iterate through the minions and decide whether to summon them or not. Since we can only have `k` minions at any time, we need to manage the minions carefully. If we have already summoned `k` minions, we need to destroy one before summoning a new one. The minion to destroy should be the one that contributes the least to the total power.

3. **Tracking Power Levels**: We need to keep track of the current power levels of the minions under control. When a new minion is summoned, we update the power levels of all existing minions by adding the `b_i` value of the new minion.

4. **Output the Sequence**: Finally, we need to output the sequence of actions (summoning and destroying) that leads to the maximum total power.

### Solution Code
```python
def solve():
    import sys
    input = sys.stdin.read
    data = input().split()
    
    idx = 0
    T = int(data[idx])
    idx += 1
    
    for _ in range(T):
        n = int(data[idx])
        k = int(data[idx+1])
        idx += 2
        
        minions = []
        for i in range(n):
            a = int(data[idx])
            b = int(data[idx+1])
            minions.append((a, b, i+1))  # storing original index as i+1
            idx += 2
        
        # Sort minions by b in descending order
        minions.sort(key=lambda x: -x[1])
        
        # Initialize the list of current minions and their power levels
        current_minions = []
        total_power = 0
        actions = []
        
        for a, b, original_idx in minions:
            if len(current_minions) < k:
                # Summon the minion
                actions.append(original_idx)
                # Update the power levels of all current minions
                for i in range(len(current_minions)):
                    current_minions[i] += b
                # Add the new minion's power
                current_minions.append(a)
                total_power += a
            else:
                # Find the minion with the smallest power to destroy
                min_power = min(current_minions)
                min_index = current_minions.index(min_power)
                # Destroy the minion
                actions.append(-(current_minions[min_index]))
                # Remove it from the list
                total_power -= current_minions[min_index]
                del current_minions[min_index]
                # Summon the new minion
                actions.append(original_idx)
                # Update the power levels of all current minions
                for i in range(len(current_minions)):
                    current_minions[i] += b
                # Add the new minion's power
                current_minions.append(a)
                total_power += a
        
        # Output the number of actions and the actions themselves
        print(len(actions))
        print(' '.join(map(str, actions)))

# Call the function to solve the problem
solve()
```

### Explanation
1. **Reading Input**: The input is read and parsed into the number of test cases, followed by the number of minions and the maximum number of minions Polycarp can control for each test case.

2. **Sorting Minions**: The minions are sorted based on their `b_i` values in descending order to maximize the boost to previously summoned minions.

3. **Summoning and Destroying**: For each minion, if the number of current minions is less than `k`, it is summoned directly. If the number of minions is already `k`, the minion with the smallest power is destroyed before summoning the new one.

4. **Tracking Power Levels**: The power levels of the minions are updated whenever a new minion is summoned. The total power is also updated accordingly.

5. **Output**: The sequence of actions (summoning and destroying) is printed for each test case.

This approach ensures that the sum of the power levels of the minions under Polycarp's control is maximized.